# Constructing a GarNet file

The goal here is to construct a file in which all of the gene and transcription factor names exist in the same namespace. We use mygene.info's API to map all the gene names to a common namespace. It isn't clear that they have the most "canonical" namespace but at present they correlate best with genecards, and seems to be more consistent than all other namespaces I know of. 

In [1]:
%pylab inline
import sys
import os
import pickle
import sqlite3
import numpy as np
import pandas as pd

known_genes_file = '../example_data/mm10/mm10_knownGene.txt'
kgXref_file = '../example_data/mm10/mm10_kgXref.txt'
motifs_file = '../example_data/mm10/motifmap.txt'

Populating the interactive namespace from numpy and matplotlib


In [2]:
sys.path.append(os.path.abspath('../src'))
from garnet import (parse_known_genes_file, parse_motifs_file, 
                    IntervalTree_from_reference, IntervalTree_from_motifs, group_by_chromosome, 
                    intersection_of_dict_of_intervaltree, save_as_pickled_object)


# Part I: Constructing a reference file

We use UCSC's [known genes](http://genome.ucsc.edu/cgi-bin/hgTables?hgsid=590772967_aCXvu74nAfyUAYeUksjLuUk1eBz3&clade=mammal&org=Human&db=hg19&hgta_group=genes&hgta_track=refGene&hgta_table=refGene&hgta_regionType=genome&position=chr21%3A33031597-33041570&hgta_outputType=primaryTable&hgta_outFileName=) and [Cross Reference (kgXref) file](http://genome.ucsc.edu/cgi-bin/hgTables?hgsid=590772967_aCXvu74nAfyUAYeUksjLuUk1eBz3&clade=mammal&org=Human&db=hg19&hgta_group=genes&hgta_track=refGene&hgta_table=kgXref&hgta_regionType=genome&position=chr21%3A33%2C031%2C597-33%2C041%2C570&hgta_outputType=primaryTable&hgta_outFileName=) as our foundation. 

In [3]:
reference = parse_known_genes_file(known_genes_file, kgXref_file, organism="mm10")
reference

,ucID,chrom,geneStrand,geneStart,geneEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,...,kgID,mRNA,spID,spDisplayID,geneName,refseq,protAcc,description,rfamAcc,tRnaName
0,uc007aet.1,chr1,-,3205903,3215632,3205903,3205903,2,"3205903,3213438,","3207317,3215632,",...,uc007aet.1,AK135172,NaN,NaN,AK135172,NaN,NaN,"Mus musculus 12 days embryo female ovary cDNA,...",NaN,NaN
1,uc007aeu.1,chr1,-,3214481,3671498,3216021,3671348,3,"3214481,3421701,3670551,","3216968,3421901,3671498,",...,uc007aeu.1,NM_001011874,Q5GH67,XKR4_MOUSE,Xkr4,NM_001011874,NP_001011874,Mus musculus X-linked Kx blood group related 4...,NaN,NaN
2,uc007aev.1,chr1,-,3648310,3658904,3648310,3648310,2,"3648310,3658846,","3650509,3658904,",...,uc007aev.1,AK149000,NaN,NaN,AK149000,NaN,NaN,Mus musculus 2 days neonate sympathetic gangli...,NaN,NaN
3,uc007aew.1,chr1,-,4290845,4409241,4292980,4409187,4,"4290845,4351909,4352201,4409169,","4293012,4352081,4352837,4409241,",...,uc007aew.1,NM_001195662,A0A0A6YXU6,A0A0A6YXU6_MOUSE,Rp1,NM_001195662,NP_001182591,Mus musculus retinitis pigmentosa 1 (human) (R...,NaN,NaN
4,uc007aex.2,chr1,-,4343506,4360314,4344599,4352825,4,"4343506,4351909,4352201,4360199,","4350091,4352081,4352837,4360314,",...,uc007aex.2,NM_011283,P56716,RP1_MOUSE,Rp1,NM_011283,NP_035413,Mus musculus retinitis pigmentosa 1 (human) (R...,NaN,NaN
5,uc007aey.1,chr1,-,4490927,4493735,4491715,4493406,2,"4490927,4493099,","4492668,4493735,",...,uc007aey.1,BC027687,Q61473,SOX17_MOUSE,Sox17,NM_001289467,NP_001276396,Mus musculus SRY (sex determining region Y)-bo...,NaN,NaN
6,uc007aez.2,chr1,-,4490927,4497354,4491715,4493406,5,"4490927,4493099,4493771,4495135,4496290,","4492668,4493466,4493863,4495942,4497354,",...,uc007aez.2,NM_011441,Q61473,SOX17_MOUSE,Sox17,NM_011441,NP_001276396,Mus musculus SRY (sex determining region Y)-bo...,NaN,NaN
7,uc007afc.2,chr1,-,4490927,4497354,4491715,4493406,4,"4490927,4493099,4493771,4496290,","4492668,4493490,4493863,4497354,",...,uc007afc.2,NM_001289464,Q61473,SOX17_MOUSE,Sox17,NM_001289464,NP_001276393,Mus musculus SRY (sex determining region Y)-bo...,NaN,NaN
8,uc033fhy.1,chr1,-,4490927,4497354,4491715,4495155,4,"4490927,4493771,4495135,4496290,","4492668,4493863,4495198,4497354,",...,uc033fhy.1,NM_001289466,A0A0A6YXS3,A0A0A6YXS3_MOUSE,Sox17,NM_001289466,NP_001276395,Mus musculus SRY (sex determining region Y)-bo...,NaN,NaN
9,uc007afa.2,chr1,-,4490927,4497354,4491715,4495155,4,"4490927,4493771,4495135,4496290,","4492668,4493863,4495942,4497354,",...,uc007afa.2,NM_001289465,A0A0A6YXS3,A0A0A6YXS3_MOUSE,Sox17,NM_001289465,NP_001276394,Mus musculus SRY (sex determining region Y)-bo...,NaN,NaN


In [4]:
reference = reference[['geneName', 'ucID', 'chrom', 'geneStrand', 'geneStart', 'geneEnd']]
reference.head()

,geneName,ucID,chrom,geneStrand,geneStart,geneEnd
0,AK135172,uc007aet.1,chr1,-,3205903,3215632
1,Xkr4,uc007aeu.1,chr1,-,3214481,3671498
2,AK149000,uc007aev.1,chr1,-,3648310,3658904
3,Rp1,uc007aew.1,chr1,-,4290845,4409241
4,Rp1,uc007aex.2,chr1,-,4343506,4360314


In [5]:
reference[reference.geneName.isnull()]

,geneName,ucID,chrom,geneStrand,geneStart,geneEnd


### ...Amazing!

In [6]:
import mygene
mg = mygene.MyGeneInfo()
df = mg.querymany(np.unique(reference.geneName.values).tolist(), scopes=['symbol', 'name', 'alias'], fields=["HGNC", "symbol"], species="mm10", as_dataframe=True, returnall=True)
df

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-28000...done.
querying 28001-29000...done.
querying 29001-30000...done.
querying 30001-31000...done.
querying 31001-32000...done.
querying 32001-32765...done.
Finished.
21336 input query terms found dup hits:
	[('0610005C13R

{'dup': [('0610005C13Rik', 2),
  ('0610007P14Rik', 2),
  ('0610009B22Rik', 2),
  ('0610009L18Rik', 2),
  ('0610009O20Rik', 2),
  ('0610010F05Rik', 2),
  ('0610012G03Rik', 2),
  ('0610031O16Rik', 2),
  ('0610037L13Rik', 2),
  ('0610038B21Rik', 2),
  ('0610039K10Rik', 2),
  ('0610040B10Rik', 2),
  ('0610040F04Rik', 2),
  ('0610043K17Rik', 2),
  ('1010001N08Rik', 2),
  ('1110002L01Rik', 2),
  ('1110004E09Rik', 2),
  ('1110006O24Rik', 2),
  ('1110008F13Rik', 2),
  ('1110008L16Rik', 3),
  ('1110008P14Rik', 2),
  ('1110015O18Rik', 2),
  ('1110019D14Rik', 2),
  ('1110020A21Rik', 2),
  ('1110028F11Rik', 2),
  ('1110028F18Rik', 2),
  ('1110032A03Rik', 2),
  ('1110036E04Rik', 2),
  ('1110037F02Rik', 2),
  ('1110038B12Rik', 2),
  ('1110046J04Rik', 2),
  ('1110051M20Rik', 2),
  ('1110059E24Rik', 2),
  ('1190003K10Rik', 2),
  ('1190005I06Rik', 3),
  ('1300002E11Rik', 2),
  ('1300017J02Rik', 2),
  ('1500004A13Rik', 3),
  ('1500009C09Rik', 2),
  ('1500011B03Rik', 2),
  ('1500011K16Rik', 2),
  ('15000

In [7]:
dup = df['dup']
missing = df['missing']
df = df['out']
df = df[["HGNC", "symbol"]].dropna()
df = df.reset_index().drop_duplicates(subset='query', keep='first').set_index('query').rename_axis(None)
df

,HGNC,symbol
4930578C19Rik,25866,CXorf36
6D,13348,LY6D
A1bg,5,A1BG
A1cf,24086,A1CF
A2m,7,A2M
A3galt2,30005,A3GALT2
A4galt,18149,A4GALT
A4gnt,17968,A4GNT
Aaas,13666,AAAS
Aacs,21298,AACS


In [8]:
len(df[df.index != df.symbol])

16486

In [9]:
missing

['5330429B09Rik',
 '5830427D03Rik',
 'A630005I04Rik',
 'AB001425',
 'AB033516',
 'AB033524',
 'AB250976',
 'AB251034',
 'AB251038',
 'AB251048',
 'AB251080',
 'AB251090',
 'AB251129',
 'AB251138',
 'AB251161',
 'AB251164',
 'AB251181',
 'AB251193',
 'AB251227',
 'AB251234',
 'AB251235',
 'AB251236',
 'AB251241',
 'AB251246',
 'AB251255',
 'AB251281',
 'AB251326',
 'AB251337',
 'AB334825',
 'AB334932',
 'AB334935',
 'AB334937',
 'AB334950',
 'AB335323',
 'AB335328',
 'AB335442',
 'AB335447',
 'AB335470',
 'AB335611',
 'AB335614',
 'AB335643',
 'AB335648',
 'AB335691',
 'AB335697',
 'AB335713',
 'AB335717',
 'AB335737',
 'AB335738',
 'AB335742',
 'AB335744',
 'AB335790',
 'AB335791',
 'AB335823',
 'AB335841',
 'AB335855',
 'AB335922',
 'AB335951',
 'AB336345',
 'AB336370',
 'AB336401',
 'AB336449',
 'AB336457',
 'AB336502',
 'AB336507',
 'AB336517',
 'AB336556',
 'AB336562',
 'AB336574',
 'AB336585',
 'AB336627',
 'AB336633',
 'AB336671',
 'AB336678',
 'AB336690',
 'AB336698',
 'AB336730

In [10]:
reference = reference.merge(df, how='left', left_on='geneName', right_index=True)
reference.head()

,geneName,ucID,chrom,geneStrand,geneStart,geneEnd,HGNC,symbol
0,AK135172,uc007aet.1,chr1,-,3205903,3215632,NaN,NaN
1,Xkr4,uc007aeu.1,chr1,-,3214481,3671498,29394,XKR4
2,AK149000,uc007aev.1,chr1,-,3648310,3658904,NaN,NaN
3,Rp1,uc007aew.1,chr1,-,4290845,4409241,10263,RP1
4,Rp1,uc007aex.2,chr1,-,4343506,4360314,10263,RP1


In [11]:
reference.symbol = reference.symbol.fillna(reference.geneName)

In [12]:
del reference['geneName']
reference.rename(columns={'symbol': 'geneName'}, inplace=True)
reference.head()

,ucID,chrom,geneStrand,geneStart,geneEnd,HGNC,geneName
0,uc007aet.1,chr1,-,3205903,3215632,NaN,AK135172
1,uc007aeu.1,chr1,-,3214481,3671498,29394,XKR4
2,uc007aev.1,chr1,-,3648310,3658904,NaN,AK149000
3,uc007aew.1,chr1,-,4290845,4409241,10263,RP1
4,uc007aex.2,chr1,-,4343506,4360314,10263,RP1


In [13]:
reference = reference[['geneName', 'chrom', 'geneStrand', 'geneStart', 'geneEnd']]
reference.head()

,geneName,chrom,geneStrand,geneStart,geneEnd
0,AK135172,chr1,-,3205903,3215632
1,XKR4,chr1,-,3214481,3671498
2,AK149000,chr1,-,3648310,3658904
3,RP1,chr1,-,4290845,4409241
4,RP1,chr1,-,4343506,4360314


In [14]:
(len(reference), len(reference.drop_duplicates()))

(63759, 53286)

In [15]:
reference.drop_duplicates(inplace=True)

In [16]:
reference.to_csv('../example_data/mm10/reference.normalized.tsv', sep='\t', index=False, header=True)

# Part II: Constructing a motifs file

Unlike hg19, this file isn't pre-normalized

In [ ]:
motifs = parse_motifs_file(motifs_file)
motifs.head()

In [3]:
# motifs = motifs[['motifName', 'chrom', 'motifStrand', 'motifStart', 'motifEnd', 'motifScore']]
motifs = pd.read_csv('../example_data/mm10/mm10_motifmap_manageable.tsv', sep='\t')
motifs.head()

,motifName,chrom,motifStrand,motifStart,motifEnd,motifScore
0,MyoD,chr1,+,3254305.0,3254317.0,0.491494
1,MyoD,chr1,-,3264187.0,3264199.0,0.290739
2,MyoD,chr1,+,3265526.0,3265538.0,0.298507
3,MyoD,chr1,-,3434478.0,3434490.0,0.526725
4,MyoD,chr1,-,3439366.0,3439378.0,0.496297


In [4]:
(len(motifs), len(motifs.drop_duplicates()))

(177781021, 176495397)

In [5]:
motifs.drop_duplicates(inplace=True)

In [6]:
motifs[motifs.motifName.isnull()]

,motifName,chrom,motifStrand,motifStart,motifEnd,motifScore


### ...Incredible!

In [8]:
import mygene
mg = mygene.MyGeneInfo()
df = mg.querymany(np.unique(motifs.motifName.values).tolist(), scopes=['symbol', 'name', 'alias'], fields=["HGNC", "symbol"], species="mm10", as_dataframe=True, returnall=True)
df

querying 1-615...done.
Finished.
549 input query terms found dup hits:
	[('AFP1', 10), ('AHR', 10), ('AIRE', 10), ('AML', 5), ('AML1', 10), ('AML2', 5), ('AML3', 5), ('AP-1
61 input query terms found no hit:
	['AML1a', 'AP-2alphaA', 'AP-2gamma', 'ATF2:c-Jun', 'AhR:Arnt', 'Barhl-1', 'CAC-binding', 'CACCC-bind


{'dup': [('AFP1', 10),
  ('AHR', 10),
  ('AIRE', 10),
  ('AML', 5),
  ('AML1', 10),
  ('AML2', 5),
  ('AML3', 5),
  ('AP-1', 10),
  ('AP-2', 20),
  ('gamma', 10),
  ('AP-2alpha', 4),
  ('AP-2rep', 3),
  ('AP-4', 10),
  ('AR', 10),
  ('AREB6', 4),
  ('ARP-1', 6),
  ('ATF-1', 10),
  ('ATF-2', 10),
  ('ATF-3', 10),
  ('ATF-4', 10),
  ('ATF1', 10),
  ('ATF2', 10),
  ('ATF3', 10),
  ('ATF4', 10),
  ('ATF5', 10),
  ('AhR', 20),
  ('Arnt', 20),
  ('HIF-1', 10),
  ('Alx-4', 10),
  ('Alx3', 10),
  ('Arx', 10),
  ('BCL6', 10),
  ('BEN', 10),
  ('BRN1', 10),
  ('BTEB3', 4),
  ('Bach1', 10),
  ('Bach2', 10),
  ('Barhl2', 10),
  ('Barx-2', 10),
  ('Barx1', 10),
  ('Blimp-1', 9),
  ('Brachyury', 10),
  ('Brn-2', 6),
  ('Brn-3c', 2),
  ('Brn-4', 3),
  ('Bsx', 10),
  ('C-MAF', 10),
  ('C-ets-1', 10),
  ('C/EBP', 20),
  ('alpha', 10),
  ('C/EBPalpha', 5),
  ('C/EBPbeta', 6),
  ('C/EBPdelta', 2),
  ('protein', 10),
  ('factor', 20),
  ('CART1', 10),
  ('CDP', 40),
  ('CR1', 10),
  ('CR3', 10),
  ('HD', 

In [9]:
dup = df['dup']
missing = df['missing']
df = df['out']
df = df[["HGNC", "symbol"]].dropna()
df = df.reset_index().drop_duplicates(subset='query', keep='first').set_index('query').rename_axis(None)
df

,HGNC,symbol
AHR,348,AHR
AIRE,360,AIRE
AML1,10471,RUNX1
AML2,10473,RUNX3
AML3,10472,RUNX2
AP-1,6205,JUNB
AP-2,11742,TFAP2A
gamma,24844,DNAJC5G
AP-2alpha,11742,TFAP2A
AP-2rep,6346,KLF12


In [10]:
len(df[df.index != df.symbol])

301

In [11]:
missing

['AML1a',
 'AP-2alphaA',
 'AP-2gamma',
 'ATF2:c-Jun',
 'AhR:Arnt',
 'Barhl-1',
 'CAC-binding',
 'CACCC-binding',
 'CEBPepsilon',
 'CHOP:C/EBPalpha',
 'CLOCK:BMAL',
 'COMP1',
 '1',
 'Dbx-1',
 'Dobox4',
 'Dobox5',
 'Ebox',
 '1',
 'FXR/RXR-alpha',
 'GABPbeta',
 'GATA-X',
 'GTF2IRD1-isoform2',
 '1',
 'HNF4alpha1',
 'Hand1:E47',
 'Ik-2',
 '(Tcfcp211)',
 'LF-A1',
 'LMAF',
 '4',
 'MEIS1A:HOXA9',
 'MEIS1B:HOXA9',
 'NF-AT2',
 'NF-AT4',
 'NKX25',
 'NRSE',
 'OCT-x',
 '1',
 'PPARalpha:RXRalpha',
 'PPARgamma:RXRalpha',
 'PPARgamma:RXRalpha',
 'Pbx-1b',
 'RORalpha2',
 'RUSH-1alpha',
 'RXR:LXR-beta',
 'SAP-1a',
 'SP1:SP3',
 '(homotetramer)',
 'TCF11:MafG',
 'TFF-1',
 'Tal-1alpha:E47',
 'Tal-1beta:E47',
 'Tal-1beta:ITF-2',
 'Tax/CREB',
 'VDR:RXR',
 'Vax-1',
 'Vax-2',
 'ZBP89',
 'aMEF-2',
 'c-Myc:Max',
 '/']

In [12]:
motifs = motifs.merge(df, how='left', left_on='motifName', right_index=True)
motifs

,motifName,chrom,motifStrand,motifStart,motifEnd,motifScore,HGNC,symbol
0,MyoD,chr1,+,3254305.0,3254317.0,0.491494,NaN,NaN
1,MyoD,chr1,-,3264187.0,3264199.0,0.290739,NaN,NaN
2,MyoD,chr1,+,3265526.0,3265538.0,0.298507,NaN,NaN
3,MyoD,chr1,-,3434478.0,3434490.0,0.526725,NaN,NaN
4,MyoD,chr1,-,3439366.0,3439378.0,0.496297,NaN,NaN
5,MyoD,chr1,-,3485328.0,3485340.0,0.526725,NaN,NaN
6,MyoD,chr1,-,3554442.0,3554454.0,0.330782,NaN,NaN
7,MyoD,chr1,+,3556737.0,3556749.0,0.330782,NaN,NaN
8,MyoD,chr1,-,3755858.0,3755870.0,0.491494,NaN,NaN
9,MyoD,chr1,-,3852371.0,3852383.0,0.474766,NaN,NaN


In [13]:
motifs.symbol = motifs.symbol.fillna(motifs.motifName)
del motifs['motifName'], motifs['HGNC']
motifs.rename(columns={'symbol': 'motifName'}, inplace=True)
motifs.head()

,chrom,motifStrand,motifStart,motifEnd,motifScore,motifName
0,chr1,+,3254305.0,3254317.0,0.491494,MyoD
1,chr1,-,3264187.0,3264199.0,0.290739,MyoD
2,chr1,+,3265526.0,3265538.0,0.298507,MyoD
3,chr1,-,3434478.0,3434490.0,0.526725,MyoD
4,chr1,-,3439366.0,3439378.0,0.496297,MyoD


In [14]:
motifs = motifs[['motifName', 'chrom', 'motifStrand', 'motifStart', 'motifEnd', 'motifScore']]
motifs.head()

,motifName,chrom,motifStrand,motifStart,motifEnd,motifScore
0,MyoD,chr1,+,3254305.0,3254317.0,0.491494
1,MyoD,chr1,-,3264187.0,3264199.0,0.290739
2,MyoD,chr1,+,3265526.0,3265538.0,0.298507
3,MyoD,chr1,-,3434478.0,3434490.0,0.526725
4,MyoD,chr1,-,3439366.0,3439378.0,0.496297


In [15]:
(len(motifs), len(motifs.drop_duplicates()))

(176495397, 176411731)

In [16]:
motifs.drop_duplicates(inplace=True)

/Users/alex/Documents/GarNet/venv/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
motifs.to_csv('../example_data/mm10/mm10_motifmap.normalized.cleaned.tsv', sep='\t', index=False, header=True)

# Part III: Merging genes and motifs

In [19]:
reference = pd.read_csv('../example_data/mm10/reference.normalized.tsv', sep='\t')
motifs = pd.read_csv('../example_data/mm10/mm10_motifmap.normalized.cleaned.tsv', sep='\t')

In [20]:
reference.head()

,geneName,chrom,geneStrand,geneStart,geneEnd
0,mKIAA1889,chr1,-,3195984,3205713
1,XKR4,chr1,-,3204562,3661579
2,AK149000,chr1,-,3638391,3648985
3,RP1,chr1,-,4280926,4399322
4,RP1,chr1,-,4333587,4350395


In [21]:
motifs.head()

,motifName,chrom,motifStrand,motifStart,motifEnd,motifScore
0,ipf1,chr1,+,9199807,9199817,1.0
1,ipf1,chr1,+,13140986,13140996,1.0
2,ipf1,chr1,-,13352952,13352962,1.0
3,ipf1,chr1,-,19764680,19764690,1.0
4,ipf1,chr1,+,23167657,23167667,1.0


In [22]:
reference = group_by_chromosome(reference)
motifs = group_by_chromosome(motifs)

In [23]:
options = {'upstream_window': 10000, 'downstream_window': 10000, 'tss': False}
reference = {chrom: IntervalTree_from_reference(genes, options) for chrom, genes in reference.items()}
motifs = {chrom: IntervalTree_from_motifs(chromosome_motifs) for chrom, chromosome_motifs in motifs.items()}

In [24]:
motifs_with_associated_genes = intersection_of_dict_of_intervaltree(motifs, reference)

02:51:42 - GarNet: INFO - Computing intersection operation of IntervalTrees for each chromosome...


In [25]:
motifs_and_genes = [{**motif, **gene} for motif, gene in motifs_with_associated_genes]
motifs_and_genes = pd.DataFrame.from_records(motifs_and_genes)
motifs_and_genes.head()

,chrom,geneEnd,geneName,geneStart,geneStrand,motifEnd,motifName,motifScore,motifStart,motifStrand
0,chrX_random,594813,Gm15093,395346,-,398121,FOSL1,0.442020,398113,+
1,chrX_random,594813,Gm15093,395346,-,522820,Dbx-2,0.660888,522804,-
2,chrX_random,594813,Gm15093,395346,-,522820,Msx-3,0.698805,522804,-
3,chrX_random,594813,Gm15093,395346,-,522820,PAX6,0.732189,522804,+
4,chrX_random,594813,Gm15093,395346,-,522820,En-1,0.787059,522804,-


In [26]:
motifs_and_genes = motifs_and_genes[['chrom','motifName','motifStrand','motifStart','motifEnd','motifScore','geneName','geneStart','geneEnd','geneStrand']]
motifs_and_genes

,chrom,motifName,motifStrand,motifStart,motifEnd,motifScore,geneName,geneStart,geneEnd,geneStrand
0,chrX_random,FOSL1,+,398113,398121,0.442020,Gm15093,395346,594813,-
1,chrX_random,Dbx-2,-,522804,522820,0.660888,Gm15093,395346,594813,-
2,chrX_random,Msx-3,-,522804,522820,0.698805,Gm15093,395346,594813,-
3,chrX_random,PAX6,+,522804,522820,0.732189,Gm15093,395346,594813,-
4,chrX_random,En-1,-,522804,522820,0.787059,Gm15093,395346,594813,-
5,chrX_random,ipf1,+,1136248,1136254,0.981493,AK185910,1141891,1144819,-
6,chrX_random,ipf1,+,1136248,1136254,0.981493,MIR138-2,1126526,1126597,-
7,chrX_random,PDX1,-,100524,100530,1.000000,SPRY3,101351,106392,-
8,chrX_random,MYOG,+,392512,392520,0.159941,Gm15093,395346,594813,-
9,chrX_random,MYEF2,+,14267,14289,0.909253,VAMP7,10737,39148,-


In [27]:
motifs_and_genes['motif_to_gene_distance'] = motifs_and_genes['motifStart'] - motifs_and_genes['geneStart']
motifs_and_genes.head()

,chrom,motifName,motifStrand,motifStart,motifEnd,motifScore,geneName,geneStart,geneEnd,geneStrand,motif_to_gene_distance
0,chrX_random,FOSL1,+,398113,398121,0.442020,Gm15093,395346,594813,-,2767
1,chrX_random,Dbx-2,-,522804,522820,0.660888,Gm15093,395346,594813,-,127458
2,chrX_random,Msx-3,-,522804,522820,0.698805,Gm15093,395346,594813,-,127458
3,chrX_random,PAX6,+,522804,522820,0.732189,Gm15093,395346,594813,-,127458
4,chrX_random,En-1,-,522804,522820,0.787059,Gm15093,395346,594813,-,127458


In [28]:
motifs_and_genes.to_csv('../example_data/mm9/intersection.tsv', sep='\t', header=True, index=False)

# Part IV: Building and exporting IntervalTrees: constructing the "GarNet File"

In [4]:
motifs_and_genes = pd.read_csv('../example_data/mm9/intersection.tsv', sep='\t')

In [5]:
motifs_and_genes = group_by_chromosome(motifs_and_genes)

In [6]:
motifs_and_genes = {chrom: IntervalTree_from_motifs(chromosome_motifs_and_genes) for chrom, chromosome_motifs_and_genes in motifs_and_genes.items()}

In [ ]:
save_as_pickled_object(motifs_and_genes, '../example_data/mm9/', 'garnet.pickle')